In [23]:
import bs4 
import requests
import pandas as pd
BASE_URL = "https://www.cityofboston.gov/assessing/search/"

In [34]:
data_folder = "data/2023"

file_name = "geocoded_merged_sales_fixed.csv"

data = pd.read_csv(f"{data_folder}/{file_name}")

# print rows that have None in any of the columns here
#  "total_room_num": None,
#             "bedrooms": None,
#             "bathrooms": None,
#             "half_bathrooms": None,
#             "kitchens": None,
#             "fireplaces": None,
#             "ac_type": None,
#             "heat_type": None,
#             "interior_condition": None,
#             "parking_spots": None,
#             "year_built": None,
#             "exterior_condition": None,
#             "foundation": None,
#             "full_address": None,
#             "owner_name": None,
#             "owner_property_count": None
noted_cols = ["total_room_num",\
            "bedrooms",\
            "bathrooms",\
            # "half_bathrooms",\
            "kitchens",\
            # "fireplaces",\
            # "ac_type",\
            # "heat_type",\
            "interior_condition",\
            "parking_spots",\
            "year_built",\
            "exterior_condition",\
            # "foundation",\
            "full_address",\
            "owner_name",\
            "owner_property_count"]

# get rows that have None in any of the columns
# data[noted_cols].isna().any(axis=1)

noted_cols.append("parcel")
# print rows that have None in any of the columns
# print the dimension of the dataframe
print(data[data[noted_cols].isna().any(axis=1)].shape)

parcels_that_have_none = data[data[noted_cols].isna().any(axis=1)]["parcel"]
data[data[noted_cols].isna().any(axis=1)][noted_cols]





(1799, 29)


,total_room_num,bedrooms,bathrooms,kitchens,interior_condition,parking_spots,year_built,exterior_condition,full_address,owner_name,owner_property_count,parcel
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02-01343-000
28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02-03084-000
32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02-00196-010
33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02-00196-020
36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02-00391-000
...,...,...,...,...,...,...,...,...,...,...,...,...
4684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04‐02447‐002
4685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17‐03777‐014
4711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04-00441-000
4712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05-03346-000


In [35]:
print(parcels_that_have_none)

23      02-01343-000
28      02-03084-000
32      02-00196-010
33      02-00196-020
36      02-00391-000
            ...     
4684    04‐02447‐002
4685    17‐03777‐014
4711    04-00441-000
4712    05-03346-000
4713    05-01637-000
Name: parcel, Length: 1799, dtype: object


In [36]:
# update the parcels_that_have_none with correct info

from crawl_sales_info import get_sales_info, process_basic_info, parse_owner_info

new_df = data.copy()

# update the dataframe 
count = 0
for parcel in parcels_that_have_none:
    soup = get_sales_info(parcel)
    info = process_basic_info(soup, False)
    # update info
    noted_cols_without_parcel = noted_cols[:-1]
    print(parcel, info["total_room_num"])
    # new_df.loc[new_df["parcel"] == parcel, noted_cols_without_parcel] = info
    # update the dataframe
    for col in info.keys():
        new_df.loc[new_df["parcel"] == parcel, col] = info[col]
    
    count += 1
    try:
        owner_name, properties = parse_owner_info(soup, True)
        if properties is not None:
            owner_property_count = len(properties)
        else:
            owner_property_count = None
        new_df.loc[new_df["parcel"] == parcel, "owner_name"] = owner_name
        new_df.loc[new_df["parcel"] == parcel, 'owner_property_count'] = owner_property_count
        print("owner updated")
    except Exception as e:
        print("no owner info available")
        continue
    print(f"Processed {count} out of {len(parcels_that_have_none)} parcels")

# save the new dataframe

new_df.to_csv(f"{data_folder}/{file_name}", index=False)



https://www.cityofboston.gov/assessing/search/?pid=02-01343-000
02-01343-000 9


/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_18422/1155061246.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '9' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_df.loc[new_df["parcel"] == parcel, col] = info[col]
/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_18422/1155061246.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_df.loc[new_df["parcel"] == parcel, col] = info[col]
/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_18422/1155061246.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3' has dtype incompatible with float64, please explicitly cast t

owner updated
Processed 1 out of 1799 parcels
https://www.cityofboston.gov/assessing/search/?pid=02-03084-000
02-03084-000 6
owner updated
Processed 2 out of 1799 parcels
https://www.cityofboston.gov/assessing/search/?pid=02-00196-010
02-00196-010 6
owner updated
Processed 3 out of 1799 parcels
https://www.cityofboston.gov/assessing/search/?pid=02-00196-020
02-00196-020 5
owner updated
Processed 4 out of 1799 parcels
https://www.cityofboston.gov/assessing/search/?pid=02-00391-000
02-00391-000 8
owner updated
Processed 5 out of 1799 parcels
https://www.cityofboston.gov/assessing/search/?pid=02-03309-000
02-03309-000 7
owner updated
Processed 6 out of 1799 parcels
https://www.cityofboston.gov/assessing/search/?pid=02-03068-000
02-03068-000 9
owner updated
Processed 7 out of 1799 parcels
https://www.cityofboston.gov/assessing/search/?pid=02-03195-000
02-03195-000 7
owner updated
Processed 8 out of 1799 parcels
https://www.cityofboston.gov/assessing/search/?pid=02-03336-000
02-03336-000 11

In [37]:
# check the new dataframe if there are any None in the noted_cols
print("Number of rows with missing values:", new_df[new_df[noted_cols].isna().any(axis=1)].shape[0])

# Display rows that still have missing values, if any
missing_rows = new_df[new_df[noted_cols].isna().any(axis=1)][noted_cols]
if not missing_rows.empty:
    print("\nRows with missing values:")
    print(missing_rows)
else:
    print("\nNo missing values found in the specified columns")


Number of rows with missing values: 4

Rows with missing values:
     total_room_num bedrooms bathrooms kitchens interior_condition  \
360               5        3         1        1               Good   
1102              3        4         1        2          Excellent   
2447              7        3         3        1               Good   
2788              5        2         2        1          Excellent   

     parking_spots year_built exterior_condition  \
360              3       2010               Good   
1102             0       1880               Good   
2447             1       1920               Good   
2788             2       2018          Excellent   

                                 full_address  \
360           63  RICHMERE RD BOSTON MA 02126   
1102      37  MONMOUTH ST # 1 BOSTON MA 02128   
2447   86  MARLBOROUGH ST # 3 BOSTON MA 02116   
2788  133  SEAPORT BLVD # 708 BOSTON MA 02210   

                                owner_name  owner_property_count        parce